> 运行前请安装dhg: `pip install git+https://github.com/iMoonLab/DeepHypergraph.git`

In [1]:
import sys
sys.path.append("../")  # 添加项目根目录到路径中

In [2]:
import torch
seed = 42
torch.manual_seed(seed)

import torch.optim as optim
from torch import nn

import dhg
from dhg import Hypergraph

import hgp
from hgp.models import HGNNP
from hgp.loss import loss_bs_matrix
from hgp.utils import from_pickle_to_hypergraph
from hgp.function import StraightThroughEstimator

DEVICE = torch.device("cuda:1") if torch.cuda.is_available() else torch.device("cpu")
DEVICE

import numpy as np
import random
torch.manual_seed(seed) # 为CPU设置随机种子
torch.cuda.manual_seed(seed) # 为当前GPU设置随机种子
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU，为所有GPU设置随机种子
np.random.seed(seed)  # Numpy module.
random.seed(seed)  # Python random module.	
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True


/home/ycq/work/graph-partition-with-gcn/.env-HGP/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from hgp.models import ParameterDict

# fmt: off
h_hyper_prmts = ParameterDict()
l_hyper_prmts = ParameterDict()

partitions = 2

"""
h_hyper_prmts["convlayers1"] = {"in_channels": 3824, "out_channels": 1024, "use_bn": False, "drop_rate": 0.4}
h_hyper_prmts["convlayers3"] = {"in_channels": 1024, "out_channels": 512, "use_bn": False, "drop_rate": 0.3}
h_hyper_prmts["convlayers4"] = {"in_channels": 512, "out_channels": 512, "use_bn": False, "drop_rate": 0.3}
h_hyper_prmts["convlayers5"] = {"in_channels": 512, "out_channels": 256, "use_bn": False, "drop_rate": 0.2}

l_hyper_prmts["linerlayer1"] = {"in_channels":list(h_hyper_prmts.values())[-1]["out_channels"], "out_channels":128, "use_bn":True, "drop_rate":0.1}
l_hyper_prmts["linerlayer2"] = {"in_channels":128, "out_channels":64, "use_bn":True, "drop_rate":0.1}
l_hyper_prmts["linerlayer3"] = {"in_channels":64, "out_channels":32, "use_bn":False, "drop_rate":0.1}
l_hyper_prmts["linerlayer4"] = {"in_channels":32, "out_channels":3, "use_bn":False, "drop_rate":0.1}
"""

h_hyper_prmts["convlayers1"] = {"in_channels": 3824, "out_channels": 2048, "use_bn": False, "drop_rate": 0.3}
h_hyper_prmts["convlayers12"] = {"in_channels": 2048, "out_channels": 1024, "use_bn": False, "drop_rate": 0.3}
h_hyper_prmts["convlayers13"] = {"in_channels": 1024, "out_channels": 1024, "use_bn": False, "drop_rate": 0.25}
h_hyper_prmts["convlayers14"] = {"in_channels": 1024, "out_channels": 512, "use_bn": False, "drop_rate": 0.2}
h_hyper_prmts["convlayers15"] = {"in_channels": 512, "out_channels": 256, "use_bn": False, "drop_rate": 0.1}
h_hyper_prmts["convlayers16"] = {"in_channels": 256, "out_channels": 128, "use_bn": False, "drop_rate": 0.1}
h_hyper_prmts["convlayers3"] = {"in_channels": 128, "out_channels": 105, "use_bn": False, "drop_rate": 0.1}
h_hyper_prmts["convlayers4"] = {"in_channels": 105, "out_channels": 128, "use_bn": False, "drop_rate": 0.1}
h_hyper_prmts["convlayers5"] = {"in_channels": 128, "out_channels": 256, "use_bn": False, "drop_rate": 0.1}
h_hyper_prmts["convlayers52"] = {"in_channels": 256, "out_channels": 512, "use_bn": False, "drop_rate": 0.1}
#h_hyper_prmts["convlayers53"] = {"in_channels": 512, "out_channels": 1024, "use_bn": False, "drop_rate": 0.1}
#h_hyper_prmts["convlayers54"] = {"in_channels": 1024, "out_channels": 512, "use_bn": False, "drop_rate": 0.1}


l_hyper_prmts["linerlayer1"] = {"in_channels":list(h_hyper_prmts.values())[-1]["out_channels"], "out_channels":956, "use_bn":True, "drop_rate":0.05}
#_hyper_prmts["linerlayer2"] = {"in_channels":3824, "out_channels":128, "use_bn":True, "drop_rate":0.05}
#l_hyper_prmts["linerlayer21"] = {"in_channels":956, "out_channels":64, "use_bn":True, "drop_rate":0.05}
#l_hyper_prmts["linerlayer3"] = {"in_channels":256, "out_channels":128, "use_bn":True, "drop_rate":0.05}
#l_hyper_prmts["linerlayer32"] = {"in_channels":128, "out_channels":64, "use_bn":True, "drop_rate":0.05}
#l_hyper_prmts["linerlayer33"] = {"in_channels":64, "out_channels":32, "use_bn":False, "drop_rate":0.05}
#l_hyper_prmts["linerlayer34"] = {"in_channels":32, "out_channels":16, "use_bn":False, "drop_rate":0.05}
l_hyper_prmts["linerlayer4"] = {"in_channels":956, "out_channels":2, "use_bn":False, "drop_rate":0.05}


hyper = {
    "h_hyper_prmts": h_hyper_prmts,
    "l_hyper_prmts":l_hyper_prmts,
    "init_features_dim":list(h_hyper_prmts.values())[0]["in_channels"],
    "partitions":partitions
}

# fmt: on

In [4]:
def loss_bs_matrix(outs, hg, device,weight):
    # fmt: off
    r"""
    对于超图的损失函数的矩阵形式.
    
        1.计算与顶点``vₙ``处于不同partition的顶点在超边``eₖ``上的数量``ne_k``.  
        2.计算与顶点``vₙ``是否处于该超边``eₖ``上.  
        3.若在,则说明``vₙ``所在的边为 **cut** , 记录该边的损失.  
    
    Args:
        ``outs``(`torch.nn.Module`):  模型的输出. Size :math:`(N, nums_classes)`.   
        ``hg``(`Hypergraph`):  超图对象.  
    """
    # fmt: on
    H = hg.H.to_dense().to(device)
    outs = outs.to(device)
    nn = torch.matmul(outs, (1 - torch.transpose(outs, 0, 1)))
    ne_k = torch.matmul(nn, H)
    ne_k = ne_k.mul(H)

    H_degree = torch.sum(H, dim=0)
    H_degree = H_degree

    H_1 = ne_k / H_degree
    a2 = 1 - H_1
    a3 = torch.prod(a2, dim=0)
    a3 = a3.sum()
    loss_1 = -1 * a3

    # pun = torch.mul(ne_k, H)

    # loss_1 = pun.sum()
    loss_2 = torch.var(torch.sum(outs, dim=0)).to(device)

    #loss = 50 * loss_1 + loss_2
    loss = weight * loss_1 + loss_2
    return loss, loss_1, loss_2

#### 定义用于训练的类Trainer

In [5]:
class Trainer(nn.Module):
    # fmt: off
    r"""
    用于承担训练的类.
    ---
    Args:
        ``net``: (``torch.nn.Module``): 网络模型.  
        ``X``: (``torch.Tensor``): 作为输入的顶点特征矩阵. Size :math:`(N, C_{in})`.  
        ``hg``: (``dhg.Hypergraph``): 包含 :math:`N` 个顶点的超图结构.  
    """
    # fmt: on
    def __init__(self, net, X, hg, optimizer):
        super().__init__()
        self.X: torch.Tensor = X.to(DEVICE)
        self.hg = hg.to(DEVICE)
        self.de = self.hg.H.to_dense().sum(dim=0).to("cpu").to(DEVICE)
        self.optimizer: torch.optim.Optimizer = optimizer
        self.layers = nn.ModuleList()
        self.layers.append(net.to(DEVICE))
        self.weight = 200
        
    def forward(self, X):
        X = self.layers[0](X, self.hg)
        for layer in self.layers[1:]:
            X = layer(X)
        return X

    def run(self, epoch):
        self.train()  # train mode | 设置为训练模式
        self.optimizer.zero_grad()
        outs = self.forward(self.X)
        loss, loss_1, loss_2 = loss_bs_matrix(outs, self.hg, device=DEVICE,weight=self.weight)
        loss.backward()
        self.optimizer.step()

        return loss.item(), loss_1.item(), loss_2.item()

#### 准备数据

In [6]:
G = from_pickle_to_hypergraph("../data/pubmed")
edges, _ = G.e
G.num_e,G.num_v

(7523, 3824)

In [7]:

X = torch.randn(size=(G.num_v, hyper["init_features_dim"]))
X = torch.eye(n=G.num_v)
net = HGNNP(hyper["h_hyper_prmts"]).to(DEVICE)
hgnn_trainer = Trainer(net=net, X=X, hg=G, optimizer=None).to(DEVICE)

for (k,v) in hyper["l_hyper_prmts"].items():
    hgnn_trainer.layers.append(nn.BatchNorm1d(num_features=v["in_channels"]).to(DEVICE)) if v["use_bn"] else None
    hgnn_trainer.layers.append(nn.ReLU().to(DEVICE))
    hgnn_trainer.layers.append(nn.Dropout(v["drop_rate"]))
    hgnn_trainer.layers.append(nn.Linear(in_features=v["in_channels"],out_features=v["out_channels"],device=DEVICE))
hgnn_trainer.layers.append(nn.Softmax(dim=1))

optim = optim.Adam(hgnn_trainer.parameters(), lr=3e-4, weight_decay=5e-8)
hgnn_trainer.optimizer = optim

In [8]:
hgnn_trainer.layers
for n,p in hgnn_trainer.named_parameters():
    print(n,p)

layers.0.layers.0.theta.weight Parameter containing:
tensor([[-0.0060, -0.0104, -0.0046,  ..., -0.0094, -0.0057, -0.0024],
        [ 0.0072,  0.0096,  0.0046,  ..., -0.0078, -0.0047,  0.0104],
        [-0.0057,  0.0120,  0.0112,  ..., -0.0066,  0.0050, -0.0148],
        ...,
        [ 0.0075, -0.0036,  0.0147,  ...,  0.0120,  0.0119, -0.0023],
        [-0.0062,  0.0061, -0.0021,  ..., -0.0094,  0.0152, -0.0124],
        [-0.0060, -0.0071, -0.0025,  ...,  0.0083, -0.0013,  0.0051]],
       device='cuda:1', requires_grad=True)
layers.0.layers.0.theta.bias Parameter containing:
tensor([ 0.0133,  0.0080, -0.0002,  ..., -0.0112, -0.0021, -0.0030],
       device='cuda:1', requires_grad=True)
layers.0.layers.1.theta.weight Parameter containing:
tensor([[-0.0209, -0.0173, -0.0123,  ...,  0.0158, -0.0184, -0.0166],
        [ 0.0017, -0.0180,  0.0122,  ..., -0.0162,  0.0025,  0.0160],
        [-0.0001,  0.0062,  0.0209,  ...,  0.0208, -0.0081, -0.0062],
        ...,
        [-0.0165, -0.0215, -0

In [9]:
hgnn_trainer.weight = 50

In [10]:
temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)
for epoch in range(6000):
    hgnn_trainer.weight = hgnn_trainer.weight - 0.006
    loss,loss_1,loss_2 = hgnn_trainer.run(epoch=epoch)
    # train
    temp_loss_total += loss
    temp_loss1 += loss_1
    temp_loss2 += loss_2
    # validation
    if epoch % 10 == 0:
        print(f"in {epoch} epoch, average loss: {temp_loss_total.item() / 10}")
        print(f"                , loss1: {temp_loss1.item() / 10}")
        print(f"                , loss2: {temp_loss2.item() / 10}")
        print(f"=================================")
        sys.stdout.flush()
        temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)

in 0 epoch, average loss: -5150.662890625
                , loss1: -106.3420166015625
                , loss2: 165.7999755859375
in 10 epoch, average loss: 32647.171875
                , loss1: -1094.99443359375
                , loss2: 87354.5
in 20 epoch, average loss: -37045.16875
                , loss1: -1066.59384765625
                , loss2: 16178.9609375
in 30 epoch, average loss: -48164.39375
                , loss1: -1062.1421875
                , loss2: 4773.846875
in 40 epoch, average loss: -50788.709375
                , loss1: -1060.7703125
                , loss2: 2017.4923828125
in 50 epoch, average loss: -52142.70625
                , loss1: -1060.1615234375
                , loss2: 569.59365234375
in 60 epoch, average loss: -52500.24375
                , loss1: -1060.10009765625
                , loss2: 145.3892822265625
in 70 epoch, average loss: -52543.10625
                , loss1: -1060.103125
                , loss2: 39.068338012695314
in 80 epoch, average loss

In [11]:
hgnn_trainer.eval()
outs = hgnn_trainer.forward(hgnn_trainer.X)
outs_straight = StraightThroughEstimator.apply(outs)
G_clone = G.clone()
edges, _  = G_clone.e
cut = 0
for vertices in edges:
    if torch.prod(outs_straight[list(vertices)], dim=0).sum() == 0:
        cut += 1
    else:
        G_clone.remove_hyperedges(vertices)
assert cut == G_clone.num_e
cut

844

In [12]:
bs = torch.sum(outs_straight, dim = 0)
#bs = torch.sum(outs, dim = 0)
bs

tensor([1912., 1912.], device='cuda:1', grad_fn=<SumBackward1>)

In [13]:
a = torch.tensor([[0.0,1.,0],[0,1,0],[0,1,0],[0,1,0],[0,1,0],[0,1,0]])
H = torch.tensor([[1.,1],[1,1],[0,1],[1,1],[0,1],[1,1]])
#H = torch.tensor([[0,1.,0,1,0,1],[1,0,1,0,1,1]])
nn = torch.matmul(a, (1 - torch.transpose(a, 0, 1)))
#ne_k = torch.matmul(nn, H)
ne_k = torch.matmul(nn, H)
ne_k = ne_k.mul(H)
H_degree = torch.sum(H, dim=0)
H_degree = H_degree
H_1 = ne_k / H_degree
    #bs = torch.where(H_1>=1)
    #print(bs)
a2 = 1 - H_1
a2 = a2.sqrt()
print('a2:',a2)
a3 = torch.prod(a2, dim=0)
print(a3)
a3 = a3.sum()
loss_1 = -1 * a3

a3

a2: tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]])
tensor([1., 1.])


tensor(2.)